# Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.



Setting up your keys
If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google.

For OpenAI, visit https://openai.com/api/
For Anthropic, visit https://console.anthropic.com/
For Google, visit https://ai.google.dev/gemini-api

When you get your API keys, you need to set them as environment variables.

In [10]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import openai
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [14]:
os.environ['OPENAI_API_KEY'] = "enter your key"
os.environ['ANTHROPIC_API_KEY'] = "enter your key"
os.environ['GOOGLE_API_KEY'] = "enter your key"

In [16]:
openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

# What information is included in the API



Typically we'll pass to the API:

1.The name of the model that should be used

2.A system message that gives overall context for the role the LLM is playing

3.A user message that provides the actual prompt

There are other parameters that can be used, including temperature which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [25]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [27]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [29]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with their computer? 

Because it had too many bytes and wasn't their type!


In [31]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [33]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the project was going to the next level!


In [37]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [39]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the statistician? 

Because they couldn't see eye to eye on the p-value! 



In [41]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution?"}
  ]

In [43]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding whether a business problem is suitable for a Large Language Model (LLM) solution involves evaluating the nature of the problem and the capabilities of LLMs. Here are some key considerations to guide your decision:

1. **Nature of the Problem**:
   - **Textual Data**: LLMs excel at tasks involving natural language. If your problem involves processing, understanding, or generating text, it might be suitable for an LLM.
   - **Complex Language Understanding**: If the problem requires understanding and generating human-like text, such as summarizing documents, generating reports, or answering questions, LLMs could be effective.
   - **Conversational Interfaces**: If the task involves building chatbots or virtual assistants that need to interact in a conversational manner, LLMs are well-suited.

2. **Task Complexity**:
   - **Creative Content Generation**: LLMs can generate creative content, such as writing articles, creating marketing copy, or drafting emails. If the problem involves this kind of task, it could be a good fit.
   - **Language Translation and Multilingual Support**: If you need to translate text between languages or provide support in multiple languages, LLMs can be beneficial.

3. **Data Availability**:
   - **Unstructured Data**: If your data is predominantly unstructured text, such as customer feedback, emails, or social media posts, LLMs can help in extracting insights or generating summaries.
   - **Training Data**: Consider whether pre-trained models are sufficient for your needs or if you require fine-tuning on specific domain data for better performance.

4. **Scalability and Cost**:
   - **Resource Intensity**: LLMs can be resource-intensive. Evaluate whether you have the computational resources and budget to deploy and maintain an LLM solution.
   - **Cost vs. Benefit**: Analyze whether the expected benefits from using an LLM outweigh the costs involved in implementing and managing it.

5. **Accuracy and Reliability**:
   - **Tolerance for Errors**: LLMs might not always produce perfect results. Assess whether your business problem can tolerate occasional errors or if high accuracy is critical.
   - **Regulatory and Compliance Needs**: Consider any industry-specific regulations that might affect the use of AI technologies.

6. **Integration with Current Systems**:
   - **Technical Feasibility**: Evaluate how well an LLM solution can integrate with your existing systems and workflows.
   - **Support for APIs and Tools**: Check if there are existing tools and APIs that can facilitate the integration of LLMs with your business processes.

7. **Ethical and Privacy Concerns**:
   - **Data Privacy**: Consider how using an LLM might impact data privacy and whether it aligns with your organization's privacy policies.
   - **Bias and Fairness**: Be aware of potential biases in LLMs and assess the impact they might have on your application.

By carefully evaluating these aspects, you can determine if a business problem is suitable for an LLM solution and plan its implementation effectively.